In [221]:
%pylab inline

import re
import xml.sax.saxutils as saxutils
import pandas as pd
from bs4 import BeautifulSoup

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras import utils, callbacks,layers, models 
from keras.preprocessing.text import Tokenizer
from multiprocessing import cpu_count

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split

import codecs


Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## initialize parameters

In [187]:
#General constants (modify them according to you environment)

# Set Numpy random seed
random.seed(1000)

# Newsline folder and format
data_folder = '/Users/xiuli/Downloads/reuters21578/' 

sgml_number_of_files = 22
sgml_file_name_template = 'reut2-NNN.sgm'

# Category files
category_files = 'all-topics-strings.lc.txt'

# Word2Vec number of features
num_features = 500
# Limit each newsline to a fixed number of words
document_max_num_words = 100


## All categories within Topic

In [189]:
# Read all categories
category_data = []

with open(data_folder + category_files, 'r') as file:
    for category in file.readlines():
        category_data.append([category.strip().lower()]) 

In [190]:
category_data #list, 135 categories

[['acq'],
 ['alum'],
 ['austdlr'],
 ['austral'],
 ['barley'],
 ['bfr'],
 ['bop'],
 ['can'],
 ['carcass'],
 ['castor-meal'],
 ['castor-oil'],
 ['castorseed'],
 ['citruspulp'],
 ['cocoa'],
 ['coconut'],
 ['coconut-oil'],
 ['coffee'],
 ['copper'],
 ['copra-cake'],
 ['corn'],
 ['corn-oil'],
 ['cornglutenfeed'],
 ['cotton'],
 ['cotton-meal'],
 ['cotton-oil'],
 ['cottonseed'],
 ['cpi'],
 ['cpu'],
 ['crude'],
 ['cruzado'],
 ['dfl'],
 ['dkr'],
 ['dlr'],
 ['dmk'],
 ['drachma'],
 ['earn'],
 ['escudo'],
 ['f-cattle'],
 ['ffr'],
 ['fishmeal'],
 ['flaxseed'],
 ['fuel'],
 ['gas'],
 ['gnp'],
 ['gold'],
 ['grain'],
 ['groundnut'],
 ['groundnut-meal'],
 ['groundnut-oil'],
 ['heat'],
 ['hk'],
 ['hog'],
 ['housing'],
 ['income'],
 ['instal-debt'],
 ['interest'],
 ['inventories'],
 ['ipi'],
 ['iron-steel'],
 ['jet'],
 ['jobs'],
 ['l-cattle'],
 ['lead'],
 ['lei'],
 ['lin-meal'],
 ['lin-oil'],
 ['linseed'],
 ['lit'],
 ['livestock'],
 ['lumber'],
 ['lupin'],
 ['meal-feed'],
 ['mexpeso'],
 ['money-fx'],
 ['mo

In [ ]:
def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)
#train_Ye=to_category_vector(train_Y) 

## Parse SGML files and Iterate all files

In [193]:
document_X = {}
document_Y = {}

document_id=[]
document_body=[]
document_train_test=[]
category=[]
document_X=pd.DataFrame()

for i in range(sgml_number_of_files):
    if i < 10:
        seq = '00' + str(i)
    else:
        seq = '0' + str(i)
        
    file_name = sgml_file_name_template.replace('NNN', seq)
    print('Reading file: %s' % file_name)
    
    with codecs.open(data_folder + file_name, 'r', encoding='utf-8',
                 errors='ignore') as file:
        content = BeautifulSoup(file.read().lower())      
        for newsline in content('reuters'):
            for c in newsline.find("topics").findAll('d'):
                      
            # News-line Id
                document_id.append(newsline['newid'])
            
            # News-line text
                document_body.append(unescape(strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')))
#            document_body.append(newsline.find("title").text)
                document_train_test.append(strip_tags(str(newsline.attrs["lewissplit"]) ))
                category.append(c.text)
        

            
document_X['document_id'] = document_id
document_X['document_body'] = document_body
document_X['train_test_label'] = document_train_test
document_X['category'] = category

                

Reading file: reut2-000.sgm
Reading file: reut2-001.sgm
Reading file: reut2-002.sgm
Reading file: reut2-003.sgm
Reading file: reut2-004.sgm
Reading file: reut2-005.sgm
Reading file: reut2-006.sgm
Reading file: reut2-007.sgm
Reading file: reut2-008.sgm
Reading file: reut2-009.sgm
Reading file: reut2-010.sgm
Reading file: reut2-011.sgm
Reading file: reut2-012.sgm
Reading file: reut2-013.sgm
Reading file: reut2-014.sgm
Reading file: reut2-015.sgm
Reading file: reut2-016.sgm
Reading file: reut2-017.sgm
Reading file: reut2-018.sgm
Reading file: reut2-019.sgm
Reading file: reut2-020.sgm
Reading file: reut2-021.sgm


## Check statistics how many articles under each category
#### could have filter categories that have few articles and strategy to balance the number of articles for each category

In [194]:
ct=document_X.groupby('category').count() #120 categories under Topics in the dataset
#Length: 14302

#document_X1=document_X [document_X.category.isin(ct[ct.document_id>10].reset_index)]   #66 left
print(ct)

                 document_id  document_body  train_test_label
category                                                     
acq                     2448           2448              2448
alum                      63             63                63
austdlr                    4              4                 4
barley                    54             54                54
bfr                        1              1                 1
bop                      116            116               116
can                        3              3                 3
carcass                   75             75                75
castor-oil                 2              2                 2
castorseed                 1              1                 1
citruspulp                 1              1                 1
cocoa                     76             76                76
coconut                    6              6                 6
coconut-oil                7              7                 7
coffee  

## Data cleansing and train/test data split

In [195]:
train_X=document_X[(document_X['train_test_label']=='train') & (document_X['category']!='')]['document_body']
test_X=document_X[(document_X['train_test_label']=='test') & (document_X['category']!='')]['document_body']

train_Y=document_X[(document_X['train_test_label']=='train') & (document_X['category']!='')]['category']
test_Y=document_X[(document_X['train_test_label']=='test') & (document_X['category']!='')]['category']

## Tokenize text

In [196]:
# Load stop-words
stop_words = set(stopwords.words('english'))

# Initialize tokenizer
# It's also possible to try with a stemmer or to mix a stemmer and a lemmatizer
tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Tokenized document collection
train_Xt = []
test_Xt = []
def tokenize(document):
    words = []

    for sentence in sent_tokenize(document):
        tokens = [lemmatizer.lemmatize(t.lower()) for t in tokenizer.tokenize(sentence) if t.lower() not in stop_words]
        words += tokens

    return words

# Tokenize


for index, row in train_X.items():
    train_Xt.append(tokenize(row))

    
for index, row in test_X.items():
    test_Xt.append(tokenize(row))
    
number_of_documents = 14302#len(document_X)


## Create new Gensim Word2Vec model

In [198]:
w2v_model = Word2Vec(train_Xt, size=num_features, min_count=1, window=10, workers=cpu_count())
w2v_model.init_sims(replace=True)
w2v_model.save(data_folder + 'reuters.word2vec')

## Vectorize each document

In [237]:
num_categories = 115
train_Xe = zeros(shape=(number_of_documents, document_max_num_words, num_features)).astype(float32)
train_ye = zeros(shape=(number_of_documents, num_categories)).astype(float32)
test_Xe = zeros(shape=(number_of_documents, document_max_num_words, num_features)).astype(float32)
test_ye = zeros(shape=(number_of_documents, num_categories)).astype(float32)

empty_word = zeros(num_features).astype(float32)

for idx, document in enumerate(train_Xt):
    for jdx, word in enumerate(document):
        if jdx == document_max_num_words:
            break
            
        else:
            if word in w2v_model:
                train_Xe[idx, jdx, :] = w2v_model[word]
            else:
                train_Xe[idx, jdx, :] = empty_word

for idx, document in enumerate(test_Xt):
    for jdx, word in enumerate(document):
        if jdx == document_max_num_words:
            break
            
        else:
            if word in w2v_model:
                test_Xe[idx, jdx, :] = w2v_model[word]
            else:
                test_Xe[idx, jdx, :] = empty_word

#train_Ye=to_category_vector(train_Y)
#test_Ye=to_category_vector(test_Y)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [238]:
train_Ye=utils.to_categorical(pd.factorize(train_Y)[0])
test_Ye=utils.to_categorical(pd.factorize(test_Y)[0])

## Model (LSTM)

In [ ]:
#Create Keras model

model = Sequential()

model.add(LSTM(int(document_max_num_words*1.5), input_shape=(document_max_num_words, num_features)))
model.add(Dropout(0.5))
model.add(Dense(num_categories))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Train and evaluate model

# Train model
model.fit(train_Xe, train_Ye, batch_size=128, nb_epoch=5, validation_data=(test_Xe, test_Ye))


In [ ]:
# Evaluate model
score, acc = model.evaluate(test_Xe, test_Ye, batch_size=128)
    
print('Score: %1.4f' % score)
print('Accuracy: %1.4f' % acc)